### SageMaker fine tune ChatGLM

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 111.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.71
    Uninstalling botocore-1.29.71:
      Successfully uninstalled botocore-1.29.71
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.71
    Uninstalling boto3-1.26.71:
      Successfully uninstalled boto3-1.26.71
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.29.116 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 743.3/743.3 kB 14.5 MB/s

In [1]:
import boto3
import sagemaker

account = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


### chatglm lora方式（单机单卡）
1: 使用羊驼语料数据  
2：语料处理，tokenization及label标注  
3：HF trainer API  
 

In [12]:
!rm -rf ./ChatGLM-Tuning
!git clone https://github.com/qingyuan18/ChatGLM-Tuning.git
!cp ./s5cmd ./ChatGLM-Tuning/

Cloning into 'ChatGLM-Tuning'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 158 (delta 83), reused 68 (delta 68), pack-reused 35 (from 1)
Receiving objects: 100% (158/158), 8.02 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (85/85), done.


## prepare docker images

In [20]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 
#From pytorch/pytorch:1.5-cuda10.1-cudnn7-runtime

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

RUN mkdir -p /opt/
COPY ./ChatGLM-Tuning/ /opt/ChatGLM-Tuning/


Overwriting Dockerfile


In [21]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


**Build image and push to ECR.**

In [22]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-chatglm-lora-demo"

In [23]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  110.6MB
Step 1/6 : From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04
 ---> 5e32024fab41
Step 2/6 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> 342d0877edf9
Step 3/6 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 1d0e79caa085
Step 4/6 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> a95c9716ab77
Step 5/6 : RUN mkdir -p /opt/
 ---> Using cache
 ---> 876c58d28008
Step 6/6 : COPY ./ChatGLM-Tuning/ /opt/ChatGLM-Tuning/
 ---> 978e526cc71d
Successfully built 978e526cc71d
Successfully tagged sagemaker-chatglm-lora-demo:latest
The push refers to repository [687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-chatglm-lora-demo]
74cf4fadd42f: Preparing
879479285a9b: Preparing
4b6461494ed0: Preparing
144ed3c716a5: Preparing
2132dc88592f: Preparing
ef7125ccaa23: Preparing
b838e4f50aeb: Preparing
ca6fb9204b2d: Preparing
89621c6faf10: Preparing
9d5ea

## Train!

In [31]:
%%writefile ./ChatGLM-Tuning/train.sh
#!/bin/bash

chmod +x ./s5cmd
pip install -r ./requirements.txt

###转化alpaca数据集为jsonl
python ./cover_alpaca2jsonl.py \
    --data_path ./data/alpaca_data.json \
    --save_path ./data/alpaca_data.jsonl 

#tokenization
python ./tokenize_dataset_rows.py \
    --jsonl_path ./data/alpaca_data.jsonl \
    --save_path ./data/alpaca \
    --max_seq_length 200

python ./finetune.py \
    --dataset_path ./data/alpaca \
    --lora_rank 8 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 1 \
    --max_steps 1000 \
    --save_steps 1000 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir /tmp/output

./s5cmd sync /tmp/output/ s3://$MODEL_S3_BUCKET/models/chatglm-lora/output/$(date +%Y-%m-%d-%H-%M-%S)/

Overwriting ./ChatGLM-Tuning/train.sh


In [32]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-chatglm-lora-demo:latest'

In [ ]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'chatglm-lora-demo'         

instance_type = 'ml.g5.12xlarge'

estimator = Estimator(role=role,
                      entry_point='train.sh',
                      source_dir='./ChatGLM-Tuning/',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      script_mode=True,
                      disable_profiler=True,
                      debugger_hook_config=False,
                      max_run=24*60*60*2)

estimator.fit()

INFO:sagemaker:Creating training-job with name: chatglm-lora-demo-2024-09-25-11-19-30-765


2024-09-25 11:19:38 Starting - Starting the training job
2024-09-25 11:19:38 Pending - Training job waiting for capacity......
2024-09-25 11:20:20 Pending - Preparing the instances for training...
2024-09-25 11:21:04 Downloading - Downloading input data...
2024-09-25 11:21:19 Downloading - Downloading the training image.....................
2024-09-25 11:25:01 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be 